In [1]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import json  


In [2]:
#Config
flat_cuttoff=1.5
percenage_maximum=75
json_path_recommendation='../PITL1-17 Recommendation System/response.json'
json_path_similarity='../PITL1-22 IncidentSimilarity/aggregate_response.json'


In [3]:
with open(json_path_recommendation) as f:
   data = json.load(f)

total_count=0
dev_count_qualified={} # contains for each analyzed input how many people are qualified for it. format is username:[len(qualified_for_issue_x),len(qualified_for_issue_y)...]  len of issue is only added to the list if the user is qualified for it

for lst in data:
    if len(lst)==0:
        print("error")
    issue_key=""
    all_reccomendations={}
    for elem in lst:
        if len(elem)>0:
            issue_key=elem["issuekey"]
            all_reccomendations[elem["username"]]=elem["score"]
    if issue_key == "":
        continue
    total_count+=1
    max_rating= max(all_reccomendations.values())
    #Filter all reccomendations to ensure only the one configured are left
    all_reccomendations = {k: v for k, v in all_reccomendations.items() if v>=flat_cuttoff and 100*v/max_rating>=percenage_maximum}
    for username in all_reccomendations.keys():
        if not username in dev_count_qualified.keys():
            dev_count_qualified[username]=[]
        dev_count_qualified[username].append(len(all_reccomendations))

In [4]:
for username,ctr_quali in dev_count_qualified.items():
    print(username, " is qualified for " ,round(100*len(ctr_quali)/total_count,2), "% of issues")
    only_qualified_ctr=len([ctr for ctr in ctr_quali if ctr==1])
    print(username, " is the only one qualified for " ,round(100*only_qualified_ctr/total_count,2), "% of issues")
    avg_other_qualified=round(np.mean(ctr_quali)-1,2)
    print( "An average of " ,avg_other_qualified, " other developers are qualified for issues he solved")



saurypande  is qualified for  67.86 % of issues
saurypande  is the only one qualified for  25.0 % of issues
An average of  1.15  other developers are qualified for issues he solved
nikanmog  is qualified for  52.14 % of issues
nikanmog  is the only one qualified for  18.57 % of issues
An average of  1.23  other developers are qualified for issues he solved
zhiyiniu  is qualified for  22.14 % of issues
zhiyiniu  is the only one qualified for  2.14 % of issues
An average of  1.87  other developers are qualified for issues he solved
ga65hif  is qualified for  40.71 % of issues
ga65hif  is the only one qualified for  4.29 % of issues
An average of  1.74  other developers are qualified for issues he solved


In [5]:
# save user_from, similarity. user_to in list

with open(json_path_similarity) as f:
   data = json.load(f)

sim_counter={}
all_users=set()
for lst in data:
    if len(lst)<2:
        print("error")
    else:
        user=lst[0]["assignee"]
        all_users.add(user)
        for elem in lst:
            assingee=elem["assignee"]
            similariy_score=float(elem["similarity_score"])
            if user not in sim_counter.keys():
                sim_counter[user]={}
            if assingee not in sim_counter[user].keys():
                sim_counter[user][assingee]=0
            sim_counter[user][assingee]+=similariy_score  
            row=[user,assingee,float(elem["similarity_score"])]
#Normalize the result
sim_counter = {k: ({k2:(v2/(v[k] if k in v.keys() else 1)) for k2,v2 in v.items()}) for k, v in sim_counter.items()}



In [6]:
all_predictions=[]

for user in all_users:
    is_qualified_for=0 #percentage value
    only_qualified_for=0 #percentage value
    avg_other_qualified=0
    if user in dev_count_qualified.keys():
        ctr_quali=dev_count_qualified[user] # List that contains  how many people are qualified for each issue that this user is qualified for.
        is_qualified_for=round(100*len(ctr_quali)/total_count,2)
        only_qualified_for=round(100*len([ctr for ctr in ctr_quali if ctr==1])/total_count,2)
        avg_other_qualified=round(np.mean(ctr_quali)-1,2)
    user_repl=[]
    #Converting the format to be consistant with early supplied json format
    for k,v in sim_counter[user].items():
        user_repl.append({k:v})
    
    all_predictions.append({"username":user,"replacements":user_repl,"only_qualified_for_percent":only_qualified_for,"is_qualified_for_percent":is_qualified_for,"avg_other_qualified":avg_other_qualified})

with open('result_similarity_and_recommendation_analysis.json', 'w') as outfile:
    json.dump(all_predictions,outfile)

all_predictions


[{'username': 'zhiyiniu',
  'replacements': [{'zhiyiniu': 1.0},
   {'saurypande': 0.3880013092881525},
   {'nikanmog': 0.31556961828015645},
   {'ga65hif': 0.07307074832052621}],
  'only_qualified_for_percent': 2.14,
  'is_qualified_for_percent': 22.14,
  'avg_other_qualified': 1.87},
 {'username': 'saurypande',
  'replacements': [{'saurypande': 1.0},
   {'nikanmog': 0.24494425406631593},
   {'zhiyiniu': 0.3102225010859598},
   {'ga65hif': 0.21830204160432454}],
  'only_qualified_for_percent': 25.0,
  'is_qualified_for_percent': 67.86,
  'avg_other_qualified': 1.15},
 {'username': 'ga65hif',
  'replacements': [{'ga65hif': 1.0},
   {'nikanmog': 0.5060666666666666},
   {'saurypande': 0.5457666666666667},
   {'zhiyiniu': 0.2896666666666667}],
  'only_qualified_for_percent': 4.29,
  'is_qualified_for_percent': 40.71,
  'avg_other_qualified': 1.74},
 {'username': 'nikanmog',
  'replacements': [{'nikanmog': 1.0},
   {'ga65hif': 0.10832242323256366},
   {'saurypande': 0.21887645798619373},
  